In [1]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

/nfshome/htseng/courses/CS203/demo/virtual_memory/notebook.py:227: SyntaxWarning: invalid escape sequence '\s'
  show = (f"[\s\*]{show}\s*\(", "^\}")
/nfshome/htseng/courses/CS203/demo/virtual_memory/notebook.py:227: SyntaxWarning: invalid escape sequence '\s'
  show = (f"[\s\*]{show}\s*\(", "^\}")
/nfshome/htseng/courses/CS203/demo/virtual_memory/notebook.py:227: SyntaxWarning: invalid escape sequence '\}'
  show = (f"[\s\*]{show}\s*\(", "^\}")


# Virtual Memory

What if we run the following program on a machine with 32GB memory?

In [2]:
render_code("memory_allocation.c")

// memory_allocation.c:1-37 (37 lines)
#define _GNU_SOURCE  
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sched.h>
#include <sys/syscall.h>
#include <time.h>

//#define dim 32768
//#define dim 49152
int main(int argc, char *argv[])
{
    int i,j;
    double *a;
    double sum=0, average;
    int dim=32768;
    if(argc < 2)
    {
        fprintf(stderr, "Usage: %s dimension\n",argv[0]);
        exit(1);
    }
    dim = atoi(argv[1]);
    a = (double *)malloc(sizeof(double)*dim*dim);
//    for(i = 0 ; i < dim; i++)
//        a[i] = (double *)malloc(sizeof(double)*dim);
    for(i = 0 ; i < dim; i++)
        for(j = 0 ; j < dim; j++)
            a[i*dim+j] = rand();
    for(i = 0 ; i < dim; i++)
        for(j = 0 ; j < dim; j++)
            sum+=a[i*dim+j];
    average = sum/(dim*dim);
    fprintf(stderr,"average: %lf\n",average);
    free(a);
    return 0;
}

In [2]:
! ssh htseng@orin-1 "free -m ; cd courses/CS203/demo/virtual_memory/; make clean; make ./memory_allocation; time ./memory_allocation 33000"

              total        used        free      shared  buff/cache   available
Mem:           7336        1370         576          50        5388        5697
Swap:         32767           4       32763
rm -f virtualization memory_allocation 
gcc -O3 memory_allocation.c -o memory_allocation
average: 1073758125.432928

real	1m5.832s
user	0m30.805s
sys	0m13.512s


What's the output of the following code if we run 4 instances at the same time?

In [14]:
render_code("virtualization.c")

// virtualization.c:1-29 (29 lines)
#define _GNU_SOURCE  
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sched.h>
#include <sys/syscall.h>
#include <time.h>
#include <stdint.h>

double a = 0;

int main(int argc, char *argv[])
{
    uint64_t i, number_of_total_processes=4, p;
    if(argc < 2)
    {
        fprintf(stderr, "Usage: %s number_of_processes\n",argv[0]);
        exit(1);
    }
    number_of_total_processes = atoi(argv[1]);
    for(i = 0; i< number_of_total_processes-1 && fork(); i++);
    srand((int)time(NULL)+(int)getpid());
    a = rand();
    fprintf(stderr, "\nProcess %d: Value of a is %lf and address of a is %p\n",(int)getpid(), a, &a);
    sleep(10);
    fprintf(stderr, "\n10 Seconds Later -- Process %d: Value of a is %lf and address of a is %p\n",(int)getpid(), a, &a);
    return 0;
}

In [15]:
! make ; ./virtualization 4

gcc -O3 virtualization.c -o virtualization

Process 909562: Value of a is 41452563.000000 and address of a is 0x5749559f3030

Process 909569: Value of a is 61346809.000000 and address of a is 0x5749559f3030

Process 909570: Value of a is 827071023.000000 and address of a is 0x5749559f3030

Process 909571: Value of a is 518873929.000000 and address of a is 0x5749559f3030

10 Seconds Later -- Process 909562: Value of a is 41452563.000000 and address of a is 0x5749559f3030

10 Seconds Later -- Process 909569: Value of a is 61346809.000000 and address of a is 0x5749559f3030

10 Seconds Later -- Process 909570: Value of a is 827071023.000000 and address of a is 0x5749559f3030

10 Seconds Later -- Process 909571: Value of a is 518873929.000000 and address of a is 0x5749559f3030


In [5]:
!sudo hugeadm --page-sizes-all

2097152
1073741824


In [6]:
!sudo hugeadm --pool-list

      Size  Minimum  Current  Maximum  Default
   2097152     1025     1025     1025        *
1073741824        0        0        0         


[Reference](https://www.intel.com/content/www/us/en/docs/vtune-profiler/cookbook/2023-0/page-faults.html)
https://www.hydrogen18.com/blog/using-hugepages-libvirt-ubuntu-jammy.html

In [9]:
pages = 2048*2048*3/(4*1024)
print(pages)

3072.0


In [16]:
! perf stat ../memory/matrix_mul/blockmm 2048 256

72676549292,19208018497,0.264295,0.213447,4.099898,0.009919,341948369,34474921141

 Performance counter stats for '../memory/matrix_mul/blockmm 2048 256':

          4,137.63 msec task-clock                       #    0.999 CPUs utilized             
                13      context-switches                 #    3.142 /sec                      
                 2      cpu-migrations                   #    0.483 /sec                      
            24,654      page-faults                      #    5.958 K/sec                     
    15,090,532,107      cpu_atom/cycles/                 #    3.647 GHz                         (0.02%)
    21,237,055,841      cpu_core/cycles/                 #    5.133 GHz                         (90.96%)
    39,893,520,392      cpu_atom/instructions/           #    2.64  insn per cycle              (0.02%)
    80,091,880,012      cpu_core/instructions/           #    5.31  insn per cycle              (90.96%)
     4,911,324,696      cpu_atom/branches/    

In [17]:
! perf stat ./run_mm.sh

72698419947,19550838407,0.268931,0.213636,4.176762,0.009844,339586907,34495372955

 Performance counter stats for './run_mm.sh':

          4,208.87 msec task-clock                       #    1.000 CPUs utilized             
                13      context-switches                 #    3.089 /sec                      
                 2      cpu-migrations                   #    0.475 /sec                      
            24,850      page-faults                      #    5.904 K/sec                     
         7,727,482      cpu_atom/cycles/                 #    0.002 GHz                         (43.13%)
    21,645,902,082      cpu_core/cycles/                 #    5.143 GHz                         (90.87%)
        11,687,939      cpu_atom/instructions/           #    1.51  insn per cycle              (62.79%)
    80,160,390,469      cpu_core/instructions/           # 10373.42  insn per cycle             (90.87%)
         3,071,738      cpu_atom/branches/               #  729.826 K/